# Initialisation

In [1199]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour Windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.optimize import curve_fit
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#import tifffile
#%matplotlib tk #pour Linux

In [1200]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [1201]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_10um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [1202]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


62

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [1203]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




# Traitement d'image

In [1204]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)


# Sélection du point à tracker

In [1205]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

In [1206]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze_x = 250
crop_sze_y = 300

Please click on the point you want to select.
Selected point: (266.57792207792215, 996.5129870129871)


Debogueur

In [1207]:
def visionneur(frame, titre):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title(titre)
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [1208]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [1209]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [ ]:
def localisateur_gaussien(intensity_grid, centre):
    x = np.arange(intensity_grid.shape[1])
    y = np.arange(intensity_grid.shape[0])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    print(max_idx)
    initial_x0 = x[max_idx[1]]
    initial_y0 = y[max_idx[0]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=5,
        sigma_y=5,
        offset=3
    )

    # Ajouter des bornes sur les paramètres x0 et y0
    params['x0'].min = 0    # Limite inférieure pour x0
    params['x0'].max = intensity_grid.shape[1] - 1  # Limite supérieure pour x0
    params['y0'].min = 0    # Limite inférieure pour y0
    params['y0'].max = intensity_grid.shape[0] - 1  # Limite supérieure pour y0

    # Ajouter des bornes sur les paramètres sigma
    params['sigma_x'].min = 1  # Limite inférieure de sigma_x
    params['sigma_x'].max = 40 # Limite supérieure de sigma_x
    params['sigma_y'].min = 1  # Limite inférieure de sigma_y
    params['sigma_y'].max = 40 # Limite supérieure de sigma_y

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    #print(f'position relative de gauss: {result.params['x0'].value}, {result.params['y0'].value}')
    
    x_position = result.params['x0'].value + centre[0] - crop_sze_x//4
    y_position = result.params['y0'].value + centre[1] - crop_sze_y//4

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [1211]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Identifier la particule et donner sa nouvelle position

In [1212]:
def empreinte_digitale(image, threshold=50, eps=3, min_samples=1):
    # 1. Observer chaque particule approximativement
    #neighborhood_size = 3  
    #local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    visionneur(image, f'particule choisie')

    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)
    size=0

    # 2. Déterminer l'empreinte digitale de notre particule
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')

            if cluster_points.shape[0]>size:
                size = cluster_points.shape[0]
                luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])

    seuil=np.max(image)*0.75
    print(f'seuil:{seuil}, taille:{size} et luminosité: {luminosity}')

    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title(f"Empreinte digitale de la particule, lum={luminosity} et taille={size}")
    plt.legend()
    plt.colorbar()  
    plt.show()

    return size, luminosity, seuil

In [1213]:
def detect_nbre_particles(image, threshold, eps=9, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        visionneur(image, f'détection du nombre de particules')

        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    # plt.imshow(image, origin='lower', cmap='gray')
    # plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            # plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    # plt.colorbar()  
    # plt.legend()
    # plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [1214]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=50, eps=9, min_samples=1):
    # 1. Obtenir le plus d'info sur la particule
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Mauvais re-crop empêche identification de la particule.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')

            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity) 
            
            #print(f'diff_taille: {size_distance} et diff_lum: {luminosity_distance}') 
            
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  
    
    #print(f'ecrat: {ecart}')
    
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Identification des particules")
    plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return position_best_particle, ecart

# Faire le crop et fit

In [1215]:
def particle_tracker_simple(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze_x // 2 + max_coords[1]
    nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

In [1216]:
#Placés ici pour pouvoir y faire référence dans particule_tracker
position_list=[[x,y]]
sigma_list = []
crop_frames = []
big_frames = []

In [ ]:
def particle_tracker(imge, x, y, taille_initiale, luminosité_initiale, seuil):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(imge)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img, seuil)

    if nb_part == 1:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)

        nouveau_x = x - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    elif  (nb_part == 0) or (nb_part == []):
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Select a point")

        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[len(position_list)-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({nouveau_x}, {nouveau_y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    else:
        imperfections, emplacements = [] , []
        for num_particule in range(nb_part):
            x_identifier=x - crop_sze_x // 2 + coordonnées[num_particule][0]
            y_identifier=y - crop_sze_y // 2 + coordonnées[num_particule][1]
            crop_pour_identifier = crop(image, x_identifier, y_identifier, crop_sze_x//2, crop_sze_x//2)
            emplacement, ecart = identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale)
            imperfections.append(ecart)
            emplacements.append(emplacement)
        qualité = np.min(imperfections)
        print(f'écart:{qualité}')
        max_coords = emplacements[np.argmin(imperfections)]  
        nouveau_x = x_identifier - crop_sze_x // 4 + max_coords[0]
        nouveau_y = y_identifier - crop_sze_y // 4 + max_coords[1]
    
    # Vérification s'il fout le camp 
    if np.sqrt((position_list[-1][0]-nouveau_x)**2+(position_list[-1][1]-nouveau_y)**2)>100:
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Problème d'identification, select a point")

        print('problème d''identification')
        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({nouveau_x}, {nouveau_y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    
    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x//2, crop_sze_y//2)    # Re-crop autour d'une seule particule


    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]
    #print(f'position estimée par le fit: {x_new}, {y_new}')

    #Affiche l'accord entre le fit gaussien et le blob (pour comparer)
    # x_range = np.arange(crop_sze_x) - crop_sze_x//2 
    # y_range = np.arange(crop_sze_y) - crop_sze_y//2
    # x_position = x_range + nouveau_x  
    # y_position = y_range + nouveau_y
    # plt.imshow(second_crop, origin='lower', cmap='gray', extent=[x_position.min(), x_position.max(), y_position.min(), y_position.max()])
    # plt.title("positionnement de la particule")
    # plt.scatter(x_new, y_new, color='r', label=f'Position estimée par fit gaussien {x_new}, {y_new}')
    # plt.scatter(nouveau_x, nouveau_y, color='r', marker='+', label=f'Position meilleure particule')
    # plt.legend()
    # plt.colorbar()  
    # plt.show()

    return [result_fit, second_crop, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [ ]:
taille_initiale, luminosité_initiale, seuil = empreinte_digitale(crop(img,x,y,crop_sze_x*0.8,crop_sze_y*0.5))
print(f'position initiale: {position_list}')

for i in range(first_frame,actual_frames):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
        print(f'frames vide:{i}')
    else:
        data = particle_tracker(img, x, y, taille_initiale, luminosité_initiale, seuil)
        position_list.append(data[2])
        x,y = position_list[i-first_frame+1]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])
    print(f'liste des positions:{position_list}')


seuil:94.5, taille:1329 et luminosité: 86.76598946576374
position initiale: [[266.57792207792215, 996.5129870129871]]
x,y:(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,

In [1219]:
# posit=[(266.1352837827377, 235.3049483666818), (265.3575908864907, 231.00866077706905), (298.49310836015366, 208.51158479040384), (267.57249098390594, 189.2576990462395), (311.4789026819993, 164.54412825058955), (299.6063061904583, 143.11453151545675)]
# # Créer une image vide de taille appropriée (par exemple, 400x400 pixels)
# image = np.zeros((1080, 1440))

# # Placer les points du vecteur 'posit' sur l'image en utilisant la valeur maximale de 1
# for (x, y) in posit:
#     # On arrondit les coordonnées pour qu'elles correspondent à des indices valides dans l'image
#     image[int(round(y)), int(round(x))] = 1

# # Afficher l'image avec imshow
# plt.imshow(image, cmap='gray', origin='lower')
# plt.title('Points sur l\'image')

# # Afficher les points avec un scatter (en superposition sur l'image)
# x_coords, y_coords = zip(*posit)
# plt.scatter(x_coords, y_coords, color='red', label='Points', marker='x')

# # Ajouter une légende
# plt.legend()

# # Afficher l'image
# plt.show()

In [1220]:
pixel_size = 3.45 # Taille absolue
f2 = 150  # Focale de L2
M_theo = 20  # Magnification of the objective
pxl = pixel_size / (f2 * M_theo / 160)  # Pixel size in um dans le plan de la particule

sigma_arr=np.array(sigma_list)* pxl * 10**(-6)
position_arr=np.array(position_list)* pxl * 10**(-6)
print(position_arr)

[[4.90503377e-05 1.83358390e-04]
 [4.92789758e-05 1.84108134e-04]
 [4.89519617e-05 1.83919771e-04]
 [4.87964972e-05 1.79933721e-04]
 [4.89450159e-05 1.78588409e-04]
 [4.92449253e-05 1.76426942e-04]
 [4.99216880e-05 1.73537663e-04]
 [5.01531250e-05 1.71739655e-04]
 [5.02945966e-05 1.69280771e-04]
 [5.01754698e-05 1.66058597e-04]
 [5.04939154e-05 1.65095890e-04]
 [5.03812957e-05 1.63351742e-04]
 [5.05349808e-05 1.60057848e-04]
 [5.07150997e-05 1.58956138e-04]
 [5.11024973e-05 1.57665801e-04]
 [5.13737705e-05 1.55047974e-04]
 [5.12821471e-05 1.52717131e-04]
 [5.12204315e-05 1.51812719e-04]
 [5.11120134e-05 1.49698586e-04]
 [5.12318498e-05 1.47042039e-04]
 [5.13236040e-05 1.45471757e-04]
 [           nan            nan]
 [           nan            nan]
 [5.15019219e-05 1.44198255e-04]
 [5.15448047e-05 1.38716606e-04]
 [5.17400512e-05 1.37139698e-04]
 [           nan            nan]
 [5.17566005e-05 1.34634079e-04]
 [           nan            nan]
 [           nan            nan]
 [5.165719

In [1221]:
def calculate_msd_with_uncertainty(position_arr, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """
    if position_arr[0:].shape[0] != delta_x[0:].shape :
        position_arr=position_arr[:len(position_arr)-1]


    msd = []
    uncertainties = []
    for d in range(1, len(position_arr)):
        diff_pairs = np.zeros_like(position_arr[d:])
        dx = np.zeros_like(delta_x[d:])
        dy = np.zeros_like(delta_y[d:])

        # Masque pour détecter les NaN
        nan_mask_pos = np.isnan(position_arr[d:]) | np.isnan(position_arr[:-d])
        nan_mask_delta = np.isnan(delta_x[d:]) | np.isnan(delta_x[:-d])
        
        # Calculer les différences uniquement où nan_mask est False
        valid_mask_pos = ~nan_mask_pos.any(axis=1)  # Inverser le masque pour obtenir les position_arr valides
        valid_mask_delta = ~nan_mask_delta
        
        diff_pairs[valid_mask_pos] = position_arr[d:][valid_mask_pos, :] - position_arr[:-d][valid_mask_pos, :]

        distances_squared = np.sum(diff_pairs**2, axis=1)
        msd.append(np.mean(distances_squared))  
        
        dx[valid_mask_delta] = delta_x[d:][valid_mask_delta] + delta_x[:-d][valid_mask_delta]
        dy[valid_mask_delta] = delta_y[d:][valid_mask_delta] + delta_y[:-d][valid_mask_delta]
        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))

    return np.array(msd), np.array(uncertainties)

calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

(array([5.55531556e-12, 1.55882213e-11, 3.28300462e-11, 5.77885122e-11,
        8.75947900e-11, 1.28205567e-10, 1.66531908e-10, 2.25379585e-10,
        2.61258598e-10, 3.30768599e-10, 3.91450046e-10, 4.67512524e-10,
        5.38614402e-10, 5.97329754e-10, 6.73345278e-10, 7.50091108e-10,
        8.27320945e-10, 9.53912366e-10, 9.98045124e-10, 1.13314458e-09,
        1.18458024e-09, 1.29928495e-09, 1.52403451e-09, 1.63236245e-09,
        1.65530749e-09, 1.94880177e-09, 1.91791097e-09, 2.28004073e-09,
        2.35007777e-09, 2.81769821e-09, 3.19525478e-09, 3.20362182e-09,
        3.48602043e-09, 4.09209347e-09, 4.23413209e-09, 4.69240518e-09,
        4.88288243e-09, 5.42935143e-09, 5.68198171e-09, 5.95168565e-09,
        6.57234144e-09, 6.86994739e-09, 7.18222229e-09, 7.48485617e-09,
        7.77679008e-09, 8.07565993e-09, 8.37382349e-09, 8.67044224e-09,
        8.92591356e-09, 9.17057975e-09, 9.44382174e-09, 9.66390739e-09,
        1.11068971e-08, 1.14645299e-08, 1.16506354e-08, 1.178535

# Résultats

In [1222]:
# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])
print('coucou')
# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(msd_values) + 1) * 0.5

# Plotting the MSD
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, msd_values, yerr=uncertainties, fmt='o', label='MSD', color='blue')

# Customize the plot
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

coucou


In [1223]:
# Fonction quadratique pour l'ajustement
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

cropped_msd = msd_values[:5]
cropped_inc = uncertainties[:5]

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(cropped_msd) + 1)

# Perform a quadratic fit using curve_fit, which gives both coefficients and covariance matrix
popt, pcov = curve_fit(quadratic, time_intervals, cropped_msd, sigma=cropped_inc)

# popt contains the optimal coefficients
coeffs = popt

# pcov is the covariance matrix, which we can use to compute uncertainties
coeff_uncertainties = np.sqrt(np.diag(pcov))  # Incertitudes sur les coefficients sont les éléments diagonaux

# Affichage des coefficients séparément
print("Coefficients du polynôme ajusté :")
print(coeffs)

# Affichage de la matrice de covariance séparément
print("Matrice de covariance des coefficients :")
print(pcov)

# Création de la fonction quadratique de l'ajustement
quadratic_fit = np.poly1d(coeffs)

# Génération des valeurs ajustées pour afficher la courbe
fitted_msd = quadratic_fit(time_intervals)

# Affichage du MSD avec les barres d'erreur
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, cropped_msd, yerr=cropped_inc, fmt='o', label='MSD', color='blue')
plt.plot(time_intervals, fitted_msd, label='Quadratic Fit', color='red', linestyle='--')

# Personnalisation du graphique
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Afficher le graphique
plt.show()

# Affichage des incertitudes sur les coefficients
print(f"Incertitudes sur les coefficients : {coeff_uncertainties}")

# Calcul du coefficient de diffusion et de la taille de la particule
r = (4 * 1.38 * 10**-23 * 300 / (6 * np.pi * 10**(-3) * coeffs[1]))  # Coefficient de diffusion
r_uncertainty = (4 * 1.38 * 10**(-23) * 300 * coeff_uncertainties[1] / (6 * np.pi * 10**(-3) * (coeffs[1])**2))  # Coefficient de diffusion

# Affichage de la taille de la particule avec incertitude
print(f"Taille de la particule (m): {r:.2e} m, avec une incertitude : {r_uncertainty:.2e}")


Coefficients du polynôme ajusté :
[ 3.47704164e-12 -2.10540379e-13  2.25779590e-12]
Matrice de covariance des coefficients :
[[ 1.56395378e-26 -8.01589820e-26  7.33944808e-26]
 [-8.01589820e-26  4.31799813e-25 -4.13076840e-25]
 [ 7.33944808e-26 -4.13076840e-25  4.35233147e-25]]
Incertitudes sur les coefficients : [1.25058138e-13 6.57114764e-13 6.59722023e-13]
Taille de la particule (m): -4.17e-06 m, avec une incertitude : 1.30e-05


In [1224]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [1225]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [1226]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()